In [1]:
from scipy.optimize import linprog
import numpy as np
import time
from utils import get_degree_cost, stochastic_block_model, is_connected

from entropicORTC import entropic_ortc
from ortc_v1 import ortc_v1
from ortc_v2 import ortc_v2
from glop_v2 import glop_v2

In [2]:
np.random.seed(10)

In [3]:
m1 = 4
m2 = 4
A1 = stochastic_block_model(np.array([m1,m1,m1,m1]), np.array([[1,0.1,0.1,0.1],[0.1,0.9,0.1,0.1],[0.1,0.1,0.8,0.1],[0.1,0.1,0.1,0.7]]))
A2 = stochastic_block_model(np.array([m2,m2,m2,m2]), np.array([[1,0.1,0.1,0.1],[0.1,0.9,0.1,0.1],[0.1,0.1,0.8,0.1],[0.1,0.1,0.1,0.7]]))
A1 = A1 / np.sum(A1)
A2 = A2 / np.sum(A2)
c = get_degree_cost(A1, A2)

eps = 0.0002

In [4]:
dx, _ = A1.shape
dy, _ = A2.shape

# d1: dx
# d2: dy
d1 = np.sum(A1, axis=1)
d2 = np.sum(A2, axis=1)

# Init
# C: dx * dy * dx * dy
C = np.exp(-(c[:, :, np.newaxis, np.newaxis] + c[np.newaxis, np.newaxis, :, :]) / eps)

# F: dx * dy * dx
F = np.tile(A1[:, np.newaxis, :], (1, dy, 1))

# G: dx * dy * dy
G = np.tile(A2, (dx, 1, 1))

# H: dx * dy * dx * dy
H = np.ones((dx, dy, dx, dy))

# K: scalar
K = np.sum(F[:, :, :, np.newaxis] * C * G[:, :, np.newaxis, :] * H)
K = 1 / K

# w: dx * dy * dx * dy    
w = C * F[:, :, :, np.newaxis] * G[:, :, np.newaxis, :] * H * K

In [28]:
edges1 = []
edge_weights1 = []
connect1 = {i: [] for i in range(dx)}
ne1 = 0
for i in range(dx):
    for j in range(i + 1, dx):
        if A1[i][j] > 0:
            edges1.append((i, j))
            edge_weights1.append(A1[i][j])
            connect1[i].append(ne1)
            ne1 += 1
            
            edges1.append((j, i))
            edge_weights1.append(A1[j][i])
            connect1[j].append(ne1)
            ne1 += 1

edges2 = []
edge_weights2 = []
connect2 = {i: [] for i in range(dy)}
ne2 = 0
for i in range(dy):
    for j in range(i + 1, dy):
        if A2[i][j] > 0:
            edges2.append((i, j))
            edge_weights2.append(A2[i][j])
            connect2[i].append(ne2)
            ne2 += 1

            edges2.append((j, i))
            edge_weights2.append(A2[j][i])
            connect2[j].append(ne2)
            ne2 += 1
            
edges1 = np.array(edges1)
edges2 = np.array(edges2)
edge_weights1 = np.array(edge_weights1)
edge_weights2 = np.array(edge_weights2)

# Init
# C: ne1 * ne2
u1_indices = edges1[:, 0].reshape(-1, 1)
v1_indices = edges2[:, 0].reshape(1, -1)
u2_indices = edges1[:, 1].reshape(-1, 1)
v2_indices = edges2[:, 1].reshape(1, -1)

In [5]:
niter = 2
eps = 0.0002
delta = 1e-8

In [6]:
w_old = np.ones((dx,dy,dx,dy))
num_iter = 0

# d: dx * dy
d = np.sum(w, axis=(2, 3))

#storing iterates and cost values
iter_history = []
cost_history = []

iter_history.append(num_iter+1)

for _ in range(niter):
    if np.max(np.abs(w - w_old)) > delta:
        num_iter += 1
        w_old = np.copy(w)

        # 2: update F
        # t: sum((dx * dy * dx * dy ) * (dx * 1 * dx * dy) * (dx * dy * dx * dy) = dx * dy * dx * dy, axis=3)
        # t: dx * dy * dx
        t = np.sum(C * G[:, :, np.newaxis, :] * H * K, axis=3)
        if num_iter == 2:
            CC = c.copy()
            GG = G.copy()
            HH = H.copy()
            KK = K
            tt = t.copy()
        # F: dx * dy * dx
        # For some reason (probably due to the limit in the float representation)
        # This way of calculating F can produce small errors ( <-1.38777878e-17) comparing to the loop
        F = (d[:, :, np.newaxis] * A1[:, np.newaxis, :] / d1[:, np.newaxis, np.newaxis]) / t
        # if num_iter == 2:
        #     FF = F.copy()
            
        # 3: update G
        # t: sum((dx * dy * dx * dy) * (dx * dy * dx * 1) * (dx * dy * dx * dy) = dx * dy * dx * dy, axis=2)
        # t: dx * dy * dy
        t = np.sum(C * F[:, :, :, np.newaxis] * H * K, axis=2)
        # G: dx * dy * dy
        # Same story with F
        G = (d[:, :, np.newaxis] * A2[np.newaxis, :, :] / d2[np.newaxis, :, np.newaxis]) / t
        # if num_iter == 2:
        #     GG = G.copy()
            
        # 4: update H
        # Create the condition
        F_nonzero = F > 0
        G_nonzero = G > 0
        cond = np.logical_and(F_nonzero[:, :, :, np.newaxis], G_nonzero[:, :, np.newaxis, :])

        # The indices where we should update H
        i, j, k, l = np.where(cond)
        # Update H base on the mask and the condition
        H[i, j, k, l] = np.sqrt((F[k, l, i] * G[k, l, j]) / (F[i, j, k] * G[i, j, l]))

        # 5: update K
        K = np.sum(F[:, :, :, np.newaxis] * C * G[:, :, np.newaxis, :] * H)
        K = 1 / K

        # 6: update w
        w = F[:, :, :, np.newaxis] * C * G[:, :, np.newaxis, :] * H * K
        
        # 7: update cost 
        d = np.sum(w, axis=(2, 3))
        if num_iter == 2:
            D = d.copy()
        exp_cost = np.sum(d * c)
        cost_history.append(exp_cost)
        iter_history.append(num_iter+1)

In [7]:
cost_history

[0.00012539244213243661, 0.00014472286295282973]

In [8]:
iter_history

[1, 2, 3]

In [14]:
np.sum(FF)

272.2172376582351

In [30]:
FF[u1_indices,:,u2_indices]

array([[[1.10054031e-02, 1.10054031e-02, 1.10054031e-02, 1.10054031e-02,
         1.52485456e-02, 1.41208031e-02, 1.27483398e-02, 1.11864408e-02,
         1.10054031e-02, 1.10054031e-02, 1.10054031e-02, 1.10054031e-02,
         1.09607483e-02, 1.40208098e-02, 1.09607483e-02, 1.09607483e-02]],

       [[2.06716173e-02, 2.06716173e-02, 2.06716173e-02, 2.06716173e-02,
         1.88171435e-02, 1.96050432e-02, 1.43939690e-02, 2.18008905e-02,
         2.06716173e-02, 2.06716173e-02, 2.06716173e-02, 2.06716173e-02,
         1.93446246e-02, 2.08420915e-02, 1.93446246e-02, 1.93446246e-02]],

       [[1.10054031e-02, 1.10054031e-02, 1.10054031e-02, 1.10054031e-02,
         1.52485456e-02, 1.41208031e-02, 1.27483398e-02, 1.11864408e-02,
         1.10054031e-02, 1.10054031e-02, 1.10054031e-02, 1.10054031e-02,
         1.09607483e-02, 1.40208098e-02, 1.09607483e-02, 1.09607483e-02]],

       [[2.15901968e-02, 2.15901968e-02, 2.15901968e-02, 2.15901968e-02,
         1.91201651e-02, 2.01487878e-02, 1

In [42]:
tt[2,4,0]

0.050824133631089397

In [39]:
tt[u1_indices,:,u2_indices]

array([[[9.71736256e-02, 9.71736256e-02, 9.71736256e-02, 9.71736256e-02,
         5.77282536e-02, 9.49690555e-02, 6.10327434e-03, 9.77681568e-02,
         9.71736256e-02, 9.71736256e-02, 9.71736256e-02, 9.71736256e-02,
         9.86136974e-02, 9.96282223e-02, 9.86136974e-02, 9.86136974e-02]],

       [[7.33930446e-02, 7.33930446e-02, 7.33930446e-02, 7.33930446e-02,
         5.08241336e-02, 7.83419509e-02, 5.85977922e-03, 6.86721357e-02,
         7.33930446e-02, 7.33930446e-02, 7.33930446e-02, 7.33930446e-02,
         7.99667272e-02, 7.60999244e-02, 7.99667272e-02, 7.99667272e-02]],

       [[9.71736256e-02, 9.71736256e-02, 9.71736256e-02, 9.71736256e-02,
         5.77282536e-02, 9.49690555e-02, 6.10327434e-03, 9.77681568e-02,
         9.71736256e-02, 9.71736256e-02, 9.71736256e-02, 9.71736256e-02,
         9.86136974e-02, 9.96282223e-02, 9.86136974e-02, 9.86136974e-02]],

       [[7.33930446e-02, 7.33930446e-02, 7.33930446e-02, 7.33930446e-02,
         5.08241336e-02, 7.83419509e-02, 5

In [40]:
edges1

array([[ 0,  1],
       [ 1,  0],
       [ 0,  2],
       [ 2,  0],
       [ 0,  3],
       [ 3,  0],
       [ 0, 12],
       [12,  0],
       [ 1,  2],
       [ 2,  1],
       [ 1,  3],
       [ 3,  1],
       [ 1, 13],
       [13,  1],
       [ 2,  3],
       [ 3,  2],
       [ 2,  4],
       [ 4,  2],
       [ 3, 11],
       [11,  3],
       [ 4,  5],
       [ 5,  4],
       [ 4,  6],
       [ 6,  4],
       [ 4,  7],
       [ 7,  4],
       [ 5,  7],
       [ 7,  5],
       [ 5, 13],
       [13,  5],
       [ 6,  7],
       [ 7,  6],
       [ 6, 12],
       [12,  6],
       [ 8, 10],
       [10,  8],
       [ 8, 11],
       [11,  8],
       [ 9, 10],
       [10,  9],
       [ 9, 11],
       [11,  9],
       [10, 11],
       [11, 10],
       [12, 13],
       [13, 12],
       [12, 14],
       [14, 12],
       [12, 15],
       [15, 12],
       [13, 15],
       [15, 13],
       [14, 15],
       [15, 14]])

In [13]:
np.sum(tt > 0)

4096